In [1]:
from dotenv import load_dotenv
import os
from enum import Enum

from langchain.output_parsers.enum import EnumOutputParser
from langchain.output_parsers import OutputFixingParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI

In [2]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY") 
print(api_key[:5])

gsk_4


In [3]:
class Sentiment(Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"

In [4]:
enum_parser = EnumOutputParser(enum=Sentiment)

llm_for_fixing = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0
)
fixing_parser = OutputFixingParser.from_llm(parser=enum_parser, llm=llm_for_fixing)

prompt = PromptTemplate(
    template=(
        "주어진 영화 리뷰의 감정을 '긍정', '부정', '보통' 중 하나로 분류하세요.\n"
        "출력은 반드시 한국어 단어 하나여야 합니다.\n\n"
        "리뷰: {review}\n"
        "감정:"
    ),
    input_variables=["review"],
)

llm = ChatOpenAI(
    api_key=api_key,
    base_url="https://api.groq.com/openai/v1",
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7, 
)

chain = prompt | llm | fixing_parser

if __name__ == "__main__":
    test_reviews = [
        "이 영화 정말 재미없어요. 시간 낭비였습니다.",
        "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
        "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
        "아이들과 보기 괜찮은 영화였습니다. 한 번쯤 볼만해요."
    ]

    for review in test_reviews:
        result = chain.invoke({"review": review})

        if isinstance(result, Sentiment):
            sentiment_label = result.value
        else:
            sentiment_label = str(result)

        print(f"리뷰: \"{review}\"")
        print(f"감정: {sentiment_label}")
        print("-" * 40)

리뷰: "이 영화 정말 재미없어요. 시간 낭비였습니다."
감정: 부정
----------------------------------------
리뷰: "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!"
감정: 긍정
----------------------------------------
리뷰: "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
감정: 보통
----------------------------------------
리뷰: "아이들과 보기 괜찮은 영화였습니다. 한 번쯤 볼만해요."
감정: 긍정
----------------------------------------
